<a href="https://colab.research.google.com/github/achinup/IPL/blob/main/IPLpredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
match = pd.read_csv('/content/matches.csv')
delivery = pd.read_csv('/content/deliveries.csv')

In [ ]:
match.head()

In [ ]:
match.shape

In [ ]:
delivery.head()

In [ ]:
delivery.shape

In [ ]:
total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

In [ ]:
total_score_df

In [ ]:
total_score_df.shape

(1528, 3)

In [ ]:
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [ ]:
total_score_df

In [ ]:
total_score_df.shape

In [ ]:
match_df = match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')#merging total score and match_id only

In [ ]:
match_df.shape

In [ ]:
match_df

In [ ]:
match_df['team1'].unique()

In [ ]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [ ]:
#Replacing the teame
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [ ]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df.shape

In [ ]:

match_df = match_df[match_df['dl_applied'] == 0]
match_df = match_df[['match_id','city','winner','total_runs']]


In [ ]:
match_df.shape

In [ ]:
delivery_df = match_df.merge(delivery,on='match_id')

In [ ]:
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [ ]:
delivery_df

In [ ]:
delivery_df.shape

In [ ]:
delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']#yin each ball run

In [ ]:
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score']#xfirst inning run

In [ ]:
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 + delivery_df['ball'])

In [ ]:
delivery_df

In [ ]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")#replacing na with "0"
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")#replacing other than na with "1"
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')#converting "0" and "1" to int
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values
delivery_df['wickets'] = 10 - wickets#number of wicket left
delivery_df.head()

In [ ]:

delivery_df.head()

In [ ]:
# crr = runs/overs
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

In [ ]:
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [ ]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0#if batting_team win than 1 else 0

In [ ]:
delivery_df['result'] = delivery_df.apply(result,axis=1)   #just calling and assigining values

In [ ]:
delivery_df.shape

(72413, 31)

In [ ]:
delivery_df.head()

In [ ]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final_df

In [ ]:
final_df = final_df.sample(final_df.shape[0])  #just shuffling rows


In [ ]:
final_df.shape

(72413, 10)

In [ ]:
final_df

In [ ]:
final_df.sample()#just get sample

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.dropna(inplace=True)#removing the null one

In [ ]:
zer0_df=final_df[final_df['balls_left'] == 0]

In [ ]:
final_df.describe()

In [ ]:
zer0_df.tail

In [ ]:
final_df = final_df[final_df['balls_left'] != 0]#19.5 thak le

In [ ]:
#Extra add
# final_df=final_df[final_df['total_runs_x']<120]

In [ ]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1 )

In [ ]:
X_train

In [ ]:
ohe=OneHotEncoder(drop='first',sparse=False)
xx=ohe.fit_transform(X_train[['batting_team','bowling_team','city']])
xx.shape

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])#D
]
,remainder='passthrough')

In [ ]:
trf

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))#D
])

In [ ]:

pipe.fit(X_train,y_train)#D

In [ ]:
y_pred = pipe.predict(X_test)#D

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)#D

0.9988662131519275

In [ ]:
pipe.predict_proba(X_test)[10]

array([3.97526219e-04, 9.99602474e-01])

In [ ]:
def match_summary(row):
    print("Batting Team-" + row['batting_team'] + " | Bowling Team-" + row['bowling_team'] + " | Target- " + str(row['total_runs_x']))


In [ ]:
def match_progression(x_df,match_id,pipe):
    match = x_df[x_df['match_id'] == match_id]
    match = match[(match['ball'] == 6)]
    temp_df = match[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr']].dropna()
    temp_df = temp_df[temp_df['balls_left'] != 0]
    result = pipe.predict_proba(temp_df)
    temp_df['lose'] = np.round(result.T[0]*100,1)
    temp_df['win'] = np.round(result.T[1]*100,1)
    temp_df['end_of_over'] = range(1,temp_df.shape[0]+1)

    target = temp_df['total_runs_x'].values[0]
    runs = list(temp_df['runs_left'].values)
    new_runs = runs[:]
    runs.insert(0,target)
    temp_df['runs_after_over'] = np.array(runs)[:-1] - np.array(new_runs)
    wickets = list(temp_df['wickets'].values)
    new_wickets = wickets[:]
    new_wickets.insert(0,10)
    wickets.append(0)
    w = np.array(wickets)
    nw = np.array(new_wickets)
    temp_df['wickets_in_over'] = (nw - w)[0:temp_df.shape[0]]

    print("Target-",target)
    temp_df = temp_df[['end_of_over','runs_after_over','wickets_in_over','lose','win']]
    return temp_df,target



In [ ]:
temp_df,target= match_progression(delivery_df,10,pipe)

temp_df


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18,8))
plt.plot(temp_df['end_of_over'],temp_df['wickets_in_over'],color='yellow',linewidth=3)
plt.plot(temp_df['end_of_over'],temp_df['win'],color='#00a65a',linewidth=4)
plt.plot(temp_df['end_of_over'],temp_df['lose'],color='red',linewidth=4)
plt.bar(temp_df['end_of_over'],temp_df['runs_after_over'])
plt.title('Target-' + str(target))

In [ ]:
teams

In [ ]:
!pip install streamlit


In [ ]:
delivery_df['city'].unique()

In [ ]:

import pickle
pickle.dump(pipe,open('score.pkl','wb'))

In [ ]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np

# Load the pre-trained model
pipe = pickle.load(open('/content/score.pkl', 'rb'))

# IPL teams and cities
teams = [
    'Sunrisers Hyderabad', 'Mumbai Indians', 'Royal Challengers Bangalore',
    'Kolkata Knight Riders', 'Kings XI Punjab', 'Chennai Super Kings',
    'Rajasthan Royals', 'Delhi Capitals'
]

cities = [
    'Hyderabad', 'Bangalore', 'Mumbai', 'Indore', 'Kolkata', 'Delhi',
    'Chandigarh', 'Jaipur', 'Chennai', 'Cape Town', 'Port Elizabeth',
    'Durban', 'Centurion', 'East London', 'Johannesburg', 'Kimberley',
    'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
    'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi', 'Abu Dhabi',
    'Sharjah', 'Mohali', 'Bengaluru'
]

# Take input from the user
batting_team = input('Select the batting team ')
bowling_team = input('Select the bowling team ')
selected_city = input('Select host city ')
total_target = int(input('Enter total target'))
current_score = int(input('Enter current score'))
wickets = int(input('Enter wickets out'))
overs_completed = int(input('Enter overs completed'))

# Calculate additional parameters
runs_left = total_target - current_score
balls_left = (20 - overs_completed) * 6
wickets_left = 10 - wickets
crr = current_score / overs_completed if overs_completed > 0 else 0
rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0

# Make a prediction
input_data = {
    'batting_team': [batting_team],
    'bowling_team': [bowling_team],
    'city': [selected_city],
    'runs_left': [runs_left],
    'balls_left': [balls_left],
    'wickets': [wickets_left],
    'total_runs_x': [total_target],
    'crr': [crr],
    'rrr': [rrr]
}



input_df = pd.DataFrame(input_data)
result = pipe.predict_proba(input_df)


print(f"Batting Team: {batting_team}")
print(f"Bowling Team: {bowling_team}")
print(f"Host City: {selected_city}")
print(f"Current Score: {current_score}")
print(f"Total Target: {total_target}")
print(f"Wickets Out: {wickets}")
print(f"Overs Completed: {overs_completed}")
print(f"CRR (Current Run Rate): {crr}")

print(f"RRR (Required Run Rate): {rrr}")
print(f"Probability of Winning bowling team is: {round(result[0][0] * 100)}%")
print(f"Probability of Winning batting team is: {round(result[0][1] * 100)}%")

